In [0]:
#Mounting the drive that contains the input image directories
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/"

'240 lab1.docx'
'255 Project'
'257 assignment excel.xlsx'
'3RD REVIEW.pptx'
 abhi.7z
 Classification_Regression_Week3-1.pdf
 CMPE180A-F18-Q1-Solution-Gomez.pdf
'CMPE 257 lab 1.docx'
'Colab Notebooks'
'Copy of F1 Visa Coaching - Persona Questions.gdoc'
'data analysis books'
 DeepLearning_Week4-1.pdf
 Dimensionality_Reduction_Week6.pdf
 Distracted_Drivers
 driver
 Ensemble_Methods_Dimensionality_Reduction_Week5-1.pdf
'Final Project.pdf'
'fire_elev (1).ipynb'
'google jam 1.jpg'
'google jam completion.jpg'
'i20 sevis fee admit letter and passport.gdoc'
'i20 sevis fee admit letter and passport.pdf'
 image001.jpg
'immunization proof.pdf'
 Introduction_Wk1-2.pdf
 kc_house_data.csv
 KC_Housing_Basic.ipynb
 KC_Housing_LinearRegression.ipynb
 KC_Housing_NN_Regression.ipynb
'Lab 1 Machine learning assignment.ipynb'
'lor latest'
'Midterm Format.pdf'
'music balvikas and drawing certificate.pdf'
'passport and visa.pdf'
'php proj'
 Resized_images
 Similarity_Proximity_Wk2-1.pdf
 Terrainwf.ipynb
 test

In [0]:
#Importing the required libraries
import numpy as np 
import pandas as pd 
import os

In [0]:
from keras import layers
from keras import models
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from os import listdir, makedirs
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, ResNet50, VGG19, InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers, regularizers
from keras.optimizers import SGD
from glob import glob
import cv2
import warnings 
warnings.filterwarnings('ignore')

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.losses import categorical_crossentropy
from keras.models import Model
from keras.models import load_model
from keras.preprocessing import image
from mpl_toolkits.axes_grid1 import ImageGrid

In [0]:
import tensorflow as tf
import time

In [0]:
root_directory ='/content/drive/My Drive/Distracted_Drivers'

In [0]:
train_directory = os.path.join(root_directory, 'train')
test_directory = os.path.join(root_directory, 'test')

In [0]:
#Finding the number of images for each class
classifications = os.listdir(train_directory)
for classes in classifications:
    if classes != '.DS_Store':
        print('{}: {} images'.format(classes, len(os.listdir(os.path.join(train_directory, classes)))))


c4: 2334 images
c2: 2325 images
c7: 2002 images
c6: 2325 images
c9: 2139 images
c1: 2269 images
c5: 2312 images
c0: 2489 images
c3: 2346 images
c8: 1911 images


In [0]:
class_list =  ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6','c7', 'c8', 'c9']
train = []
for classKey, classValue in enumerate(class_list):
    for file in os.listdir(os.path.join(train_directory, classValue)):
        train.append(['train/{}/{}'.format(classValue, file), classKey, classValue,file])
        
training_data = pd.DataFrame(train, columns=['image', 'class_key', 'class_value','image_name'])
training_data.head(10)

,image,class_key,class_value,image_name
0,train/c0/img_63126.jpg,0,c0,img_63126.jpg
1,train/c0/img_63140.jpg,0,c0,img_63140.jpg
2,train/c0/img_63152.jpg,0,c0,img_63152.jpg
3,train/c0/img_6326.jpg,0,c0,img_6326.jpg
4,train/c0/img_63314.jpg,0,c0,img_63314.jpg
5,train/c0/img_63318.jpg,0,c0,img_63318.jpg
6,train/c0/img_63345.jpg,0,c0,img_63345.jpg
7,train/c0/img_63349.jpg,0,c0,img_63349.jpg
8,train/c0/img_63389.jpg,0,c0,img_63389.jpg
9,train/c0/img_63398.jpg,0,c0,img_63398.jpg


In [0]:
class_numbers = len(np.unique(training_data["class_key"]))
print("number of classes in the given images: ", class_numbers)

number of classes in the given images:  10


In [0]:
#Performing the test train split
from sklearn.model_selection  import train_test_split
output = training_data["class_key"]

test_size = 0.30
seed = 10
X_train, X_val, y_train, y_val = train_test_split(training_data, output, test_size=test_size, random_state=seed)

In [0]:
print("Dimensions of the training dataset: {}".format(X_train.shape))
print("Dimensions of the validation dataset: {}".format(X_val.shape))

Dimensions of the training dataset: (15716, 4)
Dimensions of the validation dataset: (6736, 4)


In [0]:
#Resizing the images to desired dimension and saving it to a new directory
import cv2
from keras.preprocessing import image
def resize_images(image_location, size):
    original_image = cv2.imread(os.path.join(root_directory, image_location))
    resized_image = cv2.resize(original_image, size, interpolation = cv2.INTER_AREA)
    path = '/content/drive/My Drive/Resized_images'
    cv2.imwrite(os.path.join(path , image_location), resized_image)
    image_dimension = image.img_to_array(resized_image)
    image_dimension = np.expand_dims(image_dimension.copy(), axis=0)
    return image_dimension

In [0]:
#Finding out the dimensions of train data
dim_inp = 128

# reading image file for traing dataset
X_train_parameters = np.zeros((len(X_train), dim_inp, dim_inp, 3), dtype='float32')
for i, file in enumerate(X_train['image']):
    data_image = resize_images(file, (dim_inp, dim_inp))
    print("Training Images resized")
    X_train_parameters[i] = data_image
print('Training Images shape: {} size: {:,}'.format(X_train_parameters.shape, X_train_parameters.size))

Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images resized
Training Images 

In [0]:
#Finding out the dimensions of validation data
X_val_parameters = np.zeros((len(X_val), dim_inp, dim_inp, 3), dtype='float32')
for i, file in enumerate(X_val['image']):
    data_image = resize_images(file, (dim_inp, dim_inp))
    print("Validation Images resized")
    X_val_parameters[i] = data_image
print('Validation Images shape: {} size: {:,}'.format(X_val_parameters.shape, X_val_parameters.size))

Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation Images resized
Validation I

In [0]:
resized_train_directory= '/content/drive/My Drive/Resized_images/train'

In [0]:
#Finding out the number of images for each class in the new resized directory
classifications = os.listdir(resized_train_directory)
for classes in classifications:
    if classes != '.DS_Store':
        print('{}: {} images'.format(classes, len(os.listdir(os.path.join(resized_train_directory, classes)))))

c4: 2334 images
c3: 2346 images
c2: 2325 images
c1: 2269 images
c0: 2489 images
c9: 2139 images
c7: 2002 images
c6: 2325 images
c5: 2312 images
c8: 1911 images
